In [1]:
import sys
sys.path.append('/Users/osborne_local/projects/storms_qc/ioos_qc')

In [2]:
# Setup logging
import logging

# Notebook logging
f = logging.Formatter(
    '[%(asctime)s] %(levelname)s - %(message)s'
)
sh = logging.StreamHandler(sys.stdout)
sh.setFormatter(f)
L = logging.getLogger()
L.handlers = [sh]
L.setLevel(logging.INFO)

# Specific libraries with different logging levels
qcl = logging.getLogger('ioos_qc')
qcl.handlers = [sh]
qcl.setLevel(logging.DEBUG)
qcl.propagate = False

In [3]:
import tempfile
import os
import shutil

import pandas as pd
import numpy as np
import xarray as xr
import netCDF4 as nc4

from datetime import datetime, timedelta
from cftime import date2num, num2date
#TODO: fix aggregate method

from erddapy import ERDDAP

from ioos_qc.config import Config, NcQcConfig
from ioos_qc.streams import XarrayStream, NetcdfStream
from ioos_qc.stores import NetcdfStore, CFNetCDFStore
from ioos_qc.results import collect_results
from ioos_qc.config_creator import CreatorConfig, QcConfigCreator, QcVariableConfig, QC_CONFIG_CREATOR_SCHEMA
from ioos_qc.plotting import bokeh_plot_collected_results, bokeh_multi_plot, bokeh_plot_collected_result
from bokeh import plotting
import json

In [4]:
qc_output_path = '/Users/osborne_local/projects/storms_qc/data_output'
climatology_path = '/Users/osborne_local/projects/storms_qc/ioos_qc/resources'
# set up ERDDAP
erddap_server = 'https://ferret.pmel.noaa.gov/pmel/erddap'

# dataset_id = 'sd1054'
# # sd1054 bounds
# query_min_t = '2020-12-04T20:00:00Z'
# query_max_t = '2021-03-15T16:45:00Z'
# query_min_y = 44.7192672
# query_max_y = 47.2086944
# query_min_x = -131.1743104
# query_max_x = -126.6228352

dataset_id = 'sd1055'
# sd1055 bounds
query_min_t = '2020-09-03T17:00:00Z'
query_max_t = '2020-09-19T04:59:00Z'
query_min_y = 68.2979456
query_max_y = 72.4122944
query_min_x = -168.53312
query_max_x = -141.0582272

e = ERDDAP(server=erddap_server, protocol='tabledap')
e.response = 'ncCF'
e.dataset_id = dataset_id
e.constraints = {
    "time>=": query_min_t,
    "time<=": query_max_t,
    "latitude>=": query_min_y,
    "latitude<=": query_max_y,
    "longitude>=": query_min_x,
    "longitude<=": query_max_x
}

data_xr_full = e.to_xarray()
data_nc= e.to_ncCF()

n_obs = data_xr_full.dims['obs']
data_xr = data_xr_full.sel(obs=range(0, n_obs))
data_xr

<xarray.Dataset>
Dimensions:                         (obs: 21648, trajectory: 1)
Coordinates:
  * trajectory                      (trajectory) object '1055.0'
    latitude                        (obs) float64 ...
    longitude                       (obs) float64 ...
    time                            (obs) datetime64[ns] ...
Dimensions without coordinates: obs
Data variables:
    rowSize                         (trajectory) int32 ...
    UWND_MEAN                       (obs) float64 ...
    UWND_STDDEV                     (obs) float64 ...
    VWND_MEAN                       (obs) float64 ...
    VWND_STDDEV                     (obs) float64 ...
    WWND_MEAN                       (obs) float64 ...
    WWND_STDDEV                     (obs) float64 ...
    WIND_SPEED_MEAN                 (obs) float64 ...
    WIND_SPEED_STDDEV               (obs) float64 ...
    WIND_FROM_MEAN                  (obs) float64 ...
    WIND_FROM_STDDEV                (obs) float64 ...
    GUST_WND_MEAN                   (obs) float64 ...
    GUST_WND_STDDEV                 (obs) float64 ...
    WIND_MEASUREMENT_HEIGHT_MEAN    (obs) float64 ...
    WIND_MEASUREMENT_HEIGHT_STDDEV  (obs) float64 ...
    TEMP_AIR_MEAN                   (obs) float64 ...
    TEMP_AIR_STDDEV                 (obs) float64 ...
    RH_MEAN                         (obs) float64 ...
    RH_STDDEV                       (obs) float64 ...
    BARO_PRES_MEAN                  (obs) float64 ...
    BARO_PRES_STDDEV                (obs) float64 ...
    TEMP_IR_SEA_WING_UNCOMP_MEAN    (obs) float64 ...
    TEMP_IR_SEA_WING_UNCOMP_STDDEV  (obs) float64 ...
    WAVE_DOMINANT_PERIOD            (obs) float64 ...
    WAVE_SIGNIFICANT_HEIGHT         (obs) float64 ...
    TEMP_SBE37_MEAN                 (obs) float64 ...
    TEMP_SBE37_STDDEV               (obs) float64 ...
    SAL_SBE37_MEAN                  (obs) float64 ...
    TEMP_CTD_RBR_MEAN               (obs) float64 ...
    TEMP_CTD_RBR_STDDEV             (obs) float64 ...
Attributes:
    area:                      Arctic
    cdm_data_type:             Trajectory
    cdm_trajectory_variables:  trajectory
    Conventions:               CF-1.6, ACDD-1.3, COARDS
    creator_email:             support@saildrone.com
    creator_name:              Saildrone
    creator_url:               https://saildrone.com/
    data_mode:                 realtime
    date_created:              2020-09-19T05:30:18.013644Z
    description:               Saildrone NetCDF Format
    drone_id:                  1055
    Easternmost_Easting:       -141.0582272
    featureType:               Trajectory
    geospatial_lat_max:        72.4122944
    geospatial_lat_min:        70.2097728
    geospatial_lat_units:      degrees_north
    geospatial_lon_max:        -141.0582272
    geospatial_lon_min:        -161.5544064
    geospatial_lon_units:      degrees_east
    history:                   Saildrone\n2021-06-28T16:16:38Z (local files)\...
    id:                        131462
    infoUrl:                   https://saildrone.com/
    institution:               Saildrone
    interval:                  60.0
    keywords:                  air, air_pressure, air_temperature, arctic, at...
    keywords_vocabulary:       GCMD Science Keywords
    license:                   2019 © SAILDRONE Inc.  All Rights Reserved.  T...
    naming_authority:          com.saildrone
    netcdf_version:            4.6.3
    nodc_template_version:     NODC_NetCDF_Trajectory_Template_v2.0
    Northernmost_Northing:     72.4122944
    platform:                  Saildrone
    project:                   arctic_single_beam_mapping
    source:                    Saildrone
    sourceUrl:                 (local files)
    Southernmost_Northing:     70.2097728
    standard_name_vocabulary:  CF Standard Name Table v58
    subsetVariables:           trajectory
    summary:                   Arctic OCS Single Beam Mapping Global Telecomm...
    testOutOfDate:             now-1day
    time_covera

In [5]:
nc_filename = '/'+dataset_id+'_copy.nc'
data_xr.to_netcdf(path=qc_output_path+nc_filename)

In [6]:
# ---- generate QC config for each variable ---- #

# metadata for climatological data extraction
min_t = str(data_xr.time.min().dt.floor("D").dt.strftime("%Y-%m-%d").data)
max_t = str(data_xr.time.max().dt.ceil("D").dt.strftime("%Y-%m-%d").data)
min_x = float(data_xr.longitude.min().data)
min_y = float(data_xr.latitude.min().data)
max_x = float(data_xr.longitude.max().data)
max_y = float(data_xr.latitude.max().data)
bbox = [min_x, min_y, max_x, max_y]

# sus_min
# sus_max
# fail_min
# fail_max

# config for testing each variable
default_config = {
    'bbox': bbox,
    'start_time': min_t,
    'end_time': max_t,
    'tests': {
        'spike_test': {
            'suspect_threshold': '1',
            'fail_threshold': '2'
        },
        'gross_range_test': {
            'suspect_min': 'mean - 3 * std', #sus_min,
            'suspect_max': 'mean + 3 * std', #sus_max,
            'fail_min': 'min - 3 * std', #fail_min,
            'fail_max': 'max + 3 * std'  #fail_max
        },
        # 'aggregate': {}
    }
}

# variables to apply tests to
custom_config = {
    'air_temperature': {
        'variable': 'air'
    },
    'air_pressure': {
        'variable': 'pres'
    },
    'relative_humidity': {
        'variable': 'rhum'
    },
    'sea_water_practical_salinity': {
        'variable': 'salinity'
    },
    'eastward_wind': {
        'variable': 'uwnd'
    },
    'northward_wind': {
        'variable': 'vwnd'
    },
    'downward_wind': {
        'variable': 'wwnd'
    },
    'sea_water_temperature': {
        'variable': 'temperature'
    }
}

# generate climatology configs
creator_config = {
    'datasets': [
        {
            'name': 'ocean_atlas',
            'file_path': climatology_path+'/ocean_atlas.nc', 
            'variables': {
                'o2': 'o_an',
                'salinity': 's_an',
                'temperature': 't_an'
            },
            '3d': 'depth'
        },
        {
            'name': 'narr',
            'file_path': climatology_path+'/narr.nc', 
            'variables': {
                'air': 'air',
                'pres': 'slp',
                'rhum': 'rhum',
                'uwnd': 'uwnd',
                'vwnd': 'vwnd',
                'wwnd': 'wwnd'
            }
        }
    ]
}

cc = CreatorConfig(creator_config)
qccc = QcConfigCreator(cc)


[2021-06-28 09:38:50,888] DEBUG - Loading 2 datasets...


In [7]:
# break down variable by standard name
def not_stddev(var):
    return var and not var.endswith('SD')

air_temp = ['air_temperature']
pressure = ['air_pressure']
humidity = ['relative_humidity']
water_temp = ['sea_water_temperature']
salt = ['sea_water_practical_salinity']
u = ['eastward_wind']
v = ['northward_wind']
w = ['downward_wind']

run_tests = air_temp + pressure + humidity + water_temp + salt + u + v + w

In [8]:
final_config = {}
for var in data_xr:
    data_arr = data_xr[var]
    # skip tests for variables not specified for testing
    if 'standard_name' not in data_arr.attrs or data_arr.attrs['standard_name'] not in run_tests:
        continue
    # ignore standard deviation of variables
    if var.endswith('_STDDEV'):
        continue
    # initialize config
    config = default_config.copy() 
    # set boundary values for config
    min_t = str(data_arr.time.min().dt.floor("D").dt.strftime("%Y-%m-%d").data)
    max_t = str(data_arr.time.max().dt.ceil("D").dt.strftime("%Y-%m-%d").data)
    min_x = float(data_arr.longitude.min().data)
    min_y = float(data_arr.latitude.min().data)
    max_x = float(data_arr.longitude.max().data)
    max_y = float(data_arr.latitude.max().data)
    # add time and space boundaries to config
    bbox = [min_x, min_y, max_x, max_y]
    config['bbox'] = bbox
    config['start_time'] = min_t
    config['end_time'] = max_t
    # allow custom overrides...
    # ... on a variable name basis
    if var in custom_config:
        config.update(custom_config[var])
    # ... on a standard name basis
    if data_arr.attrs['standard_name'] in custom_config:
        config.update(custom_config[data_arr.attrs['standard_name']])
    # generate ioos_qc Config object
    qc_var = QcVariableConfig(config)
    qc_config = qccc.create_config(qc_var)
    # remove variable added by create_config
    qc_config = list(qc_config.values())[0]
    # and add it to final_config
    final_config[var] = qc_config 

final_config

[2021-06-28 09:38:52,133] DEBUG - Validating schema...
[2021-06-28 09:38:52,138] DEBUG - Subsetting uwnd by depth=0 and [-161.5544064, 70.2097728, -141.0582272, 72.4122944]...
[2021-06-28 09:38:52,143] DEBUG - Expanded bounding box to [-161.5544064, 70.2097728, -141.0582272, 72.4122944]
[2021-06-28 09:38:52,149] INFO - Used [-162.0544064, 69.7097728, -140.5582272, 72.9122944] for bounds after padding 1 times
[2021-06-28 09:38:52,150] DEBUG - Creating config...
[2021-06-28 09:38:52,170] DEBUG - Validating schema...
[2021-06-28 09:38:52,174] DEBUG - Subsetting vwnd by depth=0 and [-161.5544064, 70.2097728, -141.0582272, 72.4122944]...
[2021-06-28 09:38:52,178] DEBUG - Expanded bounding box to [-161.5544064, 70.2097728, -141.0582272, 72.4122944]
[2021-06-28 09:38:52,184] INFO - Used [-162.0544064, 69.7097728, -140.5582272, 72.9122944] for bounds after padding 1 times
[2021-06-28 09:38:52,185] DEBUG - Creating config...
[2021-06-28 09:38:52,207] DEBUG - Validating schema...
[2021-06-28 09:

{'UWND_MEAN': {'qartod': {'spike_test': {'suspect_threshold': 1.0,
    'fail_threshold': 2.0},
   'gross_range_test': {'suspect_span': [-3.8536065075572057,
     0.1265127820459977],
    'fail_span': [-5.02049537023735, 1.2721680981855938]}}},
 'VWND_MEAN': {'qartod': {'spike_test': {'suspect_threshold': 1.0,
    'fail_threshold': 2.0},
   'gross_range_test': {'suspect_span': [-2.438195829232372,
     1.6731355672192294],
    'fail_span': [-3.571842051290315, 2.5951022197489015]}}},
 'TEMP_AIR_MEAN': {'qartod': {'spike_test': {'suspect_threshold': 1.0,
    'fail_threshold': 2.0},
   'gross_range_test': {'suspect_span': [-7.6832753012539445,
     1.4869357119364737],
    'fail_span': [-11.316331431592667, 4.769619184821011]}}},
 'RH_MEAN': {'qartod': {'spike_test': {'suspect_threshold': 1.0,
    'fail_threshold': 2.0},
   'gross_range_test': {'suspect_span': [83.72031757603035, 96.9930722698103],
    'fail_span': [78.7474479743169, 100.12939313847149]}}},
 'BARO_PRES_MEAN': {'qartod': {

In [9]:
c = Config(final_config)
xs = XarrayStream(data_xr, time='time', lat='latitude', lon='longitude')
qc_results = xs.run(c)
qc_results = list(qc_results)
qc_results

[<ContextResult stream_id=UWND_MEAN>,
 <ContextResult stream_id=UWND_MEAN>,
 <ContextResult stream_id=VWND_MEAN>,
 <ContextResult stream_id=VWND_MEAN>,
 <ContextResult stream_id=TEMP_AIR_MEAN>,
 <ContextResult stream_id=TEMP_AIR_MEAN>,
 <ContextResult stream_id=RH_MEAN>,
 <ContextResult stream_id=RH_MEAN>,
 <ContextResult stream_id=BARO_PRES_MEAN>,
 <ContextResult stream_id=BARO_PRES_MEAN>,
 <ContextResult stream_id=TEMP_SBE37_MEAN>,
 <ContextResult stream_id=TEMP_SBE37_MEAN>,
 <ContextResult stream_id=SAL_SBE37_MEAN>,
 <ContextResult stream_id=SAL_SBE37_MEAN>,
 <ContextResult stream_id=TEMP_CTD_RBR_MEAN>,
 <ContextResult stream_id=TEMP_CTD_RBR_MEAN>]

In [10]:
# Save the QC results to a CF netCDF file

# Import whatever DSG class you want to save the data as. You can 
# figure out which type the original dataset was by loading it into pocean
from pocean.dsg import IncompleteMultidimensionalTrajectory

qc_out = os.path.join(qc_output_path, dataset_id + '_qc_out.nc')
qc_out_just_flags = os.path.join(qc_output_path, dataset_id + '_qc_out_just_flags.nc')

# Setup the CF store
store = CFNetCDFStore(qc_results)

# Store results with data and flags
if os.path.exists(qc_out):
    os.remove(qc_out)
qc_all = store.save(
    # The netCDF file to export OR append to
    qc_out,
    # The DSG class to save the results as
    IncompleteMultidimensionalTrajectory,
    # The QC config that was run
    c,
    # Should we write the data or just metadata? Defaults to false
    write_data=True,
    # Any kwargs to pass to the DSG class
    dsg_kwargs=dict(  
        reduce_dims=True, # Remove dimensions of size 1
        unlimited=False,  # Don't make the record dimension unlimited
        unique_dims=True  # Support loading into xarray
    )
)

# Store results only with the flags
if os.path.exists(qc_out_just_flags):
    os.remove(qc_out_just_flags)
qc_flags = store.save(
    # The netCDF file to export OR append to
    qc_out_just_flags,
    # The DSG class to save the results as
    IncompleteMultidimensionalTrajectory,
    # The QC config that was run
    c,
    # Should we write the data or just metadata? Defaults to false
    write_data=False,
    # Any kwargs to pass to the DSG class
    dsg_kwargs=dict(  
        reduce_dims=True, # Remove dimensions of size 1
        unlimited=False,  # Don't make the record dimension unlimited
        unique_dims=True  # Support loading into xarray
    )
)

L.info(qc_all)
L.info(qc_flags)

[2021-06-28 09:38:52,465] INFO - Adding column time from stream UWND_MEAN
[2021-06-28 09:38:52,468] INFO - Adding column lon from stream UWND_MEAN
[2021-06-28 09:38:52,469] INFO - Adding column lat from stream UWND_MEAN
[2021-06-28 09:38:53,352] INFO - Adding column time from stream UWND_MEAN
[2021-06-28 09:38:53,355] INFO - Adding column lon from stream UWND_MEAN
[2021-06-28 09:38:53,356] INFO - Adding column lat from stream UWND_MEAN
[2021-06-28 09:38:54,077] INFO - <class 'pocean.dsg.trajectory.im.IncompleteMultidimensionalTrajectory'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.6
    date_created: 2021-06-28T16:38:00Z
    featureType: trajectory
    cdm_data_type: Trajectory
    dimensions(sizes): obs_dim(21648)
    variables(dimensions): int32 trajectory(), float64 time(obs_dim), int32 z(obs_dim), float64 lat(obs_dim), float64 lon(obs_dim), float64 UWND_MEAN(obs_dim), uint8 UWND_MEAN_qartod_spike_test(obs_dim), uint8 UWND_MEAN_qartod_gross_range_test(o

In [11]:
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_file, output_notebook
output_notebook()

# Helper method to plot QC results using Bokeh
def plot_ncresults(ncdata, var_name, results, title, test_name):

    time = np.array(ncdata.variables['time'])
    obs = np.array(ncdata.variables[var_name])
    qc_test = results[var_name]['qartod'][test_name]

    qc_pass = np.ma.masked_where(qc_test != 1, obs)
    num_pass = (qc_test == 1).sum()
    qc_suspect = np.ma.masked_where(qc_test != 3, obs)
    num_suspect = (qc_test == 3).sum()
    qc_fail = np.ma.masked_where(qc_test != 4, obs)
    num_fail = (qc_test == 4).sum()
    qc_notrun = np.ma.masked_where(qc_test != 2, obs)

    p1 = figure(x_axis_type="datetime", 
                title=test_name + ' : ' + title + ' : p/s/f=' + str(num_pass) + '/' + str(num_suspect) + '/' +                         str(num_fail))
                
    p1.grid.grid_line_alpha=0.3
    p1.xaxis.axis_label = 'Time'
    p1.yaxis.axis_label = 'Observation Value'

    p1.line(time, obs,  legend_label='obs', color='#A6CEE3')
    p1.circle(time, qc_notrun, size=2, legend_label='qc not run', color='gray', alpha=0.2)
    p1.circle(time, qc_pass, size=4, legend_label='qc pass', color='green', alpha=0.5)
    p1.circle(time, qc_suspect, size=4, legend_label='qc suspect', color='orange', alpha=0.7)
    p1.circle(time, qc_fail, size=6, legend_label='qc fail', color='red', alpha=1.0)

    #output_file("qc.html", title="qc example")

    show(gridplot([[p1]], plot_width=800, plot_height=400))

Loading BokehJS ...

In [26]:
from bokeh.models import Legend
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_file, output_notebook
output_notebook()

# Helper method to plot QC results using Bokeh
def plot_ncresults(ncdata, var_name, title, test_name):

    time = np.array(ncdata.variables['time'])
    obs = np.array([v for v in ncdata.variables[var_name] if v is not None])
    
    # Get QC Variable names
    qc_test = ncdata.get_variables_by_attributes(
        ioos_qc_test=test_name,
        ioos_qc_target=var_name
    )[0]
    L.info(f'Plotting {qc_test.name}')
    
    qc_values = qc_test[:]
    
    qc_pass = np.ma.masked_where(qc_values != 1, obs)
    num_pass = (qc_values == 1).sum()
    qc_suspect = np.ma.masked_where(qc_values != 3, obs)
    num_suspect = (qc_values == 3).sum()
    qc_fail = np.ma.masked_where(qc_values != 4, obs)
    num_fail = (qc_values == 4).sum()
    qc_notrun = np.ma.masked_where(qc_values != 2, obs)

    p1 = figure(
        x_axis_type="datetime", 
        title=''.join((
            test_name,
            ' : ',
            title,
            ' : p/s/f=',
            str(num_pass),
            '/',
            str(num_suspect),
            '/',
            str(num_fail),
            ', ncvar=',
            qc_test.name
        ))
    )
                
    p1.grid.grid_line_alpha=0.5
    p1.xaxis.axis_label = 'Time'
    p1.yaxis.axis_label = 'Observation Value'

    obs = p1.line(time, obs, color='#A6CEE3')
    qc_not = p1.circle(time, qc_notrun, size=2, color='gray', alpha=0.2)
    qc_pass = p1.circle(time, qc_pass, size=4, color='green', alpha=0.5)
    qc_sus = p1.circle(time, qc_suspect, size=4, color='orange', alpha=0.7)
    qc_fail = p1.circle(time, qc_fail, size=6, color='red', alpha=1.0)

    legend = Legend(items=[
        ("values", [obs]),
        ("notrun", [qc_not]),
        ("pass", [qc_pass]),
        ("suspect", [qc_sus]),
        ("fail", [qc_fail]),
    ], location="top_right")

    p1.add_layout(legend, 'right')
    
    #output_file("qc.html", title="qc example")

    show(gridplot([[p1]], plot_width=800, plot_height=400))

Loading BokehJS ...

In [27]:
# ---- plot flags for QC'd variables ---- #

from bokeh.io import show

qc_var_plots = plot_ncresults(qc_all, 'UWND_MEAN', 'QC', 'gross_range_test')

# show(qc_var_plots)

[2021-06-28 09:43:37,751] INFO - Plotting UWND_MEAN_qartod_gross_range_test


In [23]:
np.array(data_xr.variables['TEMP_SBE37_MEAN'])

array([3.04  , 3.0444, 3.04  , ...,    nan,    nan,    nan])